In [ ]:
import sqlite3
import datetime
import time
from selenium import webdriver
from bs4 import BeautifulSoup

In [ ]:
conn = sqlite3.connect('./db.sqlite3')
cur = conn.cursor()

In [ ]:
browser = webdriver.Chrome('./chromedriver.exe')
browser.get('https://www.jobindexworld.com/jobpost/list')
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
for i in range(5):

    target_name = 'jobindex'
    target_url = 'https://www.jobindexworld.com/index'
    detail_target_url = None
    category_big = 'IT'
    category_middle = None
    category_small = None
    search_word = None
    
    try:
        now = datetime.datetime.now()
        create_date = now.strftime("%Y%m%d") # 스크래핑 날짜

        # click해서 들어간 창
        browser.find_elements_by_css_selector('article.cpn-circle-conts-item')[i].click() # 클릭 페이지
        time.sleep(1)
        html_scraping = browser.page_source 
        soup_scraping = BeautifulSoup(html_scraping, 'html.parser')

        tags_signs = soup_scraping.select('div.circle-conts-view-main-conts-box-question > div') # '◈' 기호와 if문을 통해 form을 선별

        recruit = soup_scraping.select('article.circle-conts-view > h3.tit-h3') # 주제, 제목
        recruit_title = recruit[0].text.strip()

        uri = soup_scraping.select('meta[property="og:url"]') # 구직 회사 내용 화면
        detail_uri = uri[0]['content']

        start_date = soup_scraping.select('div.date')
        start_date_t = start_date[0].text.strip()[1:9] # 날짜, 시간 -> data time: https://docs.python.org/ko/3/library/datetime.html
        apply_start_date = '20'+start_date_t.replace('.','')
        
        
    #---------------------------------------------------------------------------------------------------            
        # 윗태그 2개 아래태그 5개 스크래핑 양식(career_requirements 컬럼이 없으므로 ' ' 추가)
        if len(tags_signs) > 9:
            tags_li = soup_scraping.select('div.circle-conts-view-main-conts-box-question > div > ul > li')
            tags_li_data = []

            for a in range(len(tags_li)):
                tags_li_data.append(tags_li[a].text.strip().replace(' ','').replace('\n','').split(':'))

            for b in range(len(tags_li_data)):
                if tags_li_data[b][0] == '회사개요':
                    company_name = tags_li_data[b][1]
                elif tags_li_data[b][0] == '모집기간':
                    apply_end_date = tags_li_data[b][1]
            

            tags = soup_scraping.select('div.circle-conts-view-main-conts-box-question > div')
            tags = tags[1:len(tags)]
            tags_data = []

            for c in range(len(tags)):
                tags_data.append(tags[c].text.strip().replace('◈ ',''))

            for d in range(len(tags_data)):
                if tags_data[d] == '직무 개요':
                    task = tags_data[d+1]

                elif tags_data[d] == '경력 요건':
                    need_career = tags_data[d+1]

                elif tags_data[d] == '처우':
                    team_env = tags_data[d+1]

                elif tags_data[d] == '제출 서류 및 기타':
                    need_doc = tags_data[d+1]

            career_requirements = None
            print(i,'페이지' ,'1번 스크래핑 양식')

    #-------------------------------------------------------------------------------------------------
        # 윗태그 5개 스크래핑 양식(task, need_doc이 없으므로 ' '을 추가)
        else:
            tags_li = soup_scraping.select('div.circle-conts-view-main-conts-box-question > div > ul > li')
            tags_li_data = []

            for e in range(len(tags_li)):
                tags_li_data.append(tags_li[e].text.strip().replace(' ','').replace('\n','').split(':'))

            for f in range(len(tags_li_data)):
                if tags_li_data[f][0] == '회사개요':
                    company_name = tags_li_data[f][1]

                elif tags_li_data[f][0] == '경력레벨':
                    need_career = tags_li_data[f][1]

                elif tags_li_data[f][0] == '급여수준':
                    team_env = tags_li_data[f][1]

                elif tags_li_data[f][0] == '교육수준':
                    career_requirements = tags_li_data[f][1]

                elif tags_li_data[f][0] == '모집기간':
                    apply_end_date = tags_li_data[f][1]


            task = None
            need_doc = None
            print(i,'페이지','2번 스크래핑 양식')

    #---------------------------------------------------------------------------------------------------       

        contact = soup_scraping.select('span.writer-info') # 담당컨설턴트
        contact_people = contact[0].text.strip()[0:3] # 이름 뽑아내기

        hash_list = []
        hash = soup_scraping.select('div.keyword-wrap > a.keyword') # 해시태그
        for j in range(len(hash)):  # 반복문으로 해시태그들을 뽑아 리스트로 만들기
            hash_t = hash[j].text.strip()
            hash_list.append(hash_t)
        
        hashtag = ''
        for k in hash_list:
            hashtag = hashtag + k
            
        print(id, target_name, target_url, detail_target_url, category_big, category_middle, category_small, search_word, create_date, recruit_title, company_name, detail_uri, apply_start_date, apply_end_date, task, need_career, team_env, need_doc, contact_people, hashtag)    
        time.sleep(1)
        cur.execute('''insert into SCRAPPING_SITE_B(
            apply_end_date,
            apply_start_date,
            career_requirements,
            category_big,
            company_name,
            contact_people,
            detail_uri,
            hashtag,
            need_career,
            need_doc,
            recruit_title,
            target_name,
            target_url,
            task,
            team_env)
        values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', (apply_end_date, apply_start_date, career_requirements, category_big, company_name, contact_people, detail_uri, hashtag, need_career, need_doc, recruit_title, target_name, target_url, task, team_env))
        conn.commit()
        browser.back()
        # 더보기 button 클릭하기 태그 : i.icon-plus-sign-to-add
        
    except:
        pass
# conn.close()

In [ ]:
cur.execute("SELECT * FROM SCRAPPING_SITE_B") # newspaper text(기사)의 내용을 추출

articles = [] # list 선언

for row in cur.fetchall():
  articles.append(row) 
articles

In [ ]:
conn.close()